In [12]:
import requests
import csv

In [10]:
def getFootballUrl(league,season):
    return f"https://www.football-data.co.uk/mmz4281/{season}/{league}.csv"

def downloadData(league, season):
    url = getFootballUrl(league, season)
    r = requests.get(url)
    with open(f'../data/{league}-{season}.csv', 'wb') as f:
        f.write(r.content)

In [11]:
downloadData("E0",1920)

In [16]:
with open("../data/E0-1920.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        print(row['HomeTeam'])

Liverpool
West Ham
Bournemouth
Burnley
Crystal Palace
Watford
Tottenham
Leicester
Newcastle
Man United
Arsenal
Aston Villa
Brighton
Everton
Norwich
Southampton
Man City
Sheffield United
Chelsea
Wolves
Aston Villa
Norwich
Brighton
Man United
Sheffield United
Watford
Liverpool
Bournemouth
Tottenham
Wolves
Southampton
Chelsea
Crystal Palace
Leicester
Man City
Newcastle
West Ham
Burnley
Everton
Arsenal
Liverpool
Brighton
Man United
Sheffield United
Tottenham
Wolves
Norwich
Bournemouth
Watford
Aston Villa
Southampton
Leicester
Burnley
Everton
Man City
Newcastle
Crystal Palace
West Ham
Arsenal
Chelsea
Sheffield United
Aston Villa
Bournemouth
Chelsea
Crystal Palace
Tottenham
Wolves
Everton
Leicester
Man United
Brighton
Burnley
Liverpool
Norwich
Watford
West Ham
Arsenal
Man City
Southampton
Newcastle
Everton
Aston Villa
Bournemouth
Chelsea
Leicester
Tottenham
Wolves
Crystal Palace
Man United
Sheffield United
Southampton
Man City
Brighton
Watford
West Ham
Burnley
Newcastle
Arsenal
Liverpool
Nor